In [3]:
import os
import opensmile
import pandas as pd
import numpy as np
from natsort import natsorted
import pandas as pd
from pydub import AudioSegment as AS
from ChaFile import *
from difflib import SequenceMatcher as sm
import editdistance as ed
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from collections import Counter

In [4]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

dir_healthy_wav = input("enter the (full) path to your directory of healthy WAV files --> ")
dir_healthy_cha = input("enter the (full) path to your directory of healthy CHA files --> ")

dir_AD_wav = input("enter the (full) path to your directory of AD WAV files --> ")
dir_AD_cha = input("enter the (full) path to your directory of AD CHA files --> ")

dir_MCI_wav = input("enter the (full) path to your directory of MCI WAV files --> ")
dir_MCI_cha = input("enter the (full) path to your directory of MCI CHA files --> ")


In [5]:
def checker(wav, cha):
    files_from_dir = os.listdir(wav)
    files_from_dir_2 = os.listdir(cha)
    if len(files_from_dir) != len(files_from_dir_2):
        quit("not equal")

checker(dir_healthy_wav, dir_healthy_cha)
checker(dir_AD_wav, dir_AD_cha)
checker(dir_MCI_wav, dir_MCI_cha)

AD_cha_files = natsorted(os.listdir(dir_AD_cha))
AD_wav_files = natsorted(os.listdir(dir_AD_wav))

MCI_cha_files = natsorted(os.listdir(dir_MCI_cha))
MCI_wav_files = natsorted(os.listdir(dir_MCI_wav))

Healthy_cha_files = natsorted(os.listdir(dir_healthy_cha))
Healthy_wav_files = natsorted(os.listdir(dir_healthy_wav))

In [6]:
keys_1 = ["command_1", "command_2", "command_3", "command_4", "command_5", "command_6", "command_7", "command_8", "command_9", 
"command_10", "command_11", "command_12", "command_13", "command_14", "command_15", "command_16", "command_17", "command_18", "command_19",
"command_20", "command_21", "command_22", "command_23", "command_24", "command_25", "command_26", "command_27", "command_28", "command_29",
"command_30", "command_31", "command_32", "command_33"]

COMMANDS = ["Alexa what is the weather outside ? ",
"Alexa what is today's date ? ", 
"Alexa what time is it ? ", 
"Alexa when is Thanksgiving ? ",
"Alexa how do you bake chocolate chip cookies ? ",
"Alexa what is two times four ? ",
"Alexa how many tablespoons in a cup ? ",
"Alexa how do you spell symptom ? ",
"Alexa play classical music . ",
"Alexa volume eight . ", 
"Alexa play jazz . ",
"Alexa volume six . ",
"Alexa stop . ",
"Alexa remind me to start the laundry tomorrow at two p_m . ",
"Alexa remind me to feed the dog at seven p_m everyday . ",
"Alexa tell me my reminders . ",
"Alexa remember my daughter's birthday is June first . ", 
"Alexa set a timer in five seconds . ",
"Alexa set my alarm for seven a_m tomorrow . ",
"Alexa add oranges and grapes to my shopping list . ",
"Alexa what is in my shopping list ? ",
"Alexa call six zero three six six zero two two zero three . ",
"Alexa hang up . ",
"Alexa find my phone . ",
"Alexa quit . ",
"Alexa turn the bedroom light on . ",
"Alexa turn the bedroom light red . ",
"Alexa change brightness to ten . ",
"Alexa turn off the bedroom light . ",
"Alexa open the kitchen camera . ",
"Alexa hide the kitchen camera . ",
"Alexa play white collar on fire tv . ",
"Alexa pause . ",
"yes . "]


In [175]:
def distance(str1, str2):
    sql = sm(None, str1, str2)
    return sql.ratio()


def categorize(cha_dir, cha_file, ft: list):
    os.chdir(cha_dir)
    cha = ChaFile(cha_file)
    frame = pd.DataFrame()
    lines = cha.getLines()

    min_list = []
    closest_commands = []
    utter_list = []
    ur_count = 0

    for data in lines:
        utterance = data["xol"]
        utterance = utterance.split(" ")
        dist = []
        edit_dist = []
        if utterance[0][0] == "[":
            # change to if it contains []
            # frame["UNRECOGNIZED " + str(ur_count)] = utterance, "N/A", "N/A"
            ur_count += 1
        else:
            utterance = data["xol"]
            for com in COMMANDS:
                dist.append(distance(com, data["xol"]))
                edit_dist.append(ed.eval(com, utterance))
            edit_min = min(edit_dist)
            closest_neighbor = COMMANDS[dist.index(max(dist))]
            closest_commands.append(closest_neighbor)
            utter_list.append(utterance)
            min_list.append(edit_min)
    num = Counter(closest_commands)
    print(cha_file)
    # print(len(utter_list), len(closest_commands), len(min_list))
    for i in num:
        multi_count = 0
        sin_count = 0
        if num[i] > 1:
            #print("multiple ", i, "number of times:", num[i])
            min_catcher = []
            com_catcher = []
            for g in closest_commands:
                if g != i:
                    multi_count += 1
                else:
                    min_catcher.append(min_list[multi_count])
                    com_catcher.append(utter_list[multi_count])
                    multi_count += 1
            aver = np.mean(min_catcher)
            frame[i] = com_catcher, aver, num[i] 
            print("Multi", com_catcher, aver, num[i])
        else:
            for g in closest_commands:
                if g != i:
                    sin_count += 1
                else:
                    frame[i] = i, min_list[sin_count], num[i]
                    print("Single", i, min_list[sin_count], num[i])
    os.chdir("..")
    with pd.ExcelWriter("editdist_2_test.xlsx", mode="a", engine="openpyxl") as writer:
        frame.to_excel(writer, sheet_name=cha_file)  
    ft.append(frame.loc[1:2].values.tolist())
    ft.append(ur_count)

        


                    print("closest command, shown by index:", closest_commands[count])
                    print("actual command: ", utter_list[count])
                    print("and the edit distance, shown by index: ", min_list[count])

In [176]:
ad_ft = []
for i in range(len(AD_cha_files)):
    categorize(dir_AD_cha, AD_cha_files[i], ad_ft)

mci_ft = []
for i in range(len(MCI_cha_files)):
    categorize(dir_MCI_cha, MCI_cha_files[i], mci_ft)

healthy_ft = []
for i in range(len(Healthy_cha_files)):
    categorize(dir_healthy_cha, Healthy_cha_files[i], healthy_ft)

012.cha
Single Alexa what is the weather outside ?  0 1
Single Alexa what is today's date ?  0 1
Multi ['Alexa what time is it ? ', 'Alexa what is my gi ? '] 3.5 2
Single Alexa when is Thanksgiving ?  0 1
Single Alexa how do you bake chocolate chip cookies ?  0 1
Single Alexa what is two times four ?  0 1
Single Alexa how many tablespoons in a cup ?  0 1
Single Alexa how do you spell symptom ?  0 1
Single Alexa play classical music .  0 1
Single Alexa volume eight .  3 1
Multi ['Alexa stop . ', 'Alexa stop . ', 'I get it Alexa . ', 'Alexa stop . '] 3.0 4
Single Alexa play jazz .  0 1
Single Alexa volume six .  0 1
Single Alexa remind me to start the laundry tomorrow at two p_m .  0 1
Single Alexa remind me to feed the dog at seven p_m everyday .  0 1
Single Alexa tell me my reminders .  0 1
Single Alexa remember my daughter's birthday is June first .  0 1
Single yes .  0 1
Multi ['Alexa set a timer in five . ', 'Alexa five seconds . '] 8.5 2
Single Alexa set my alarm for seven a_m tomo

In [181]:
def flatten(ft):
    ft_flat = []
    for f in range(len(ft)):
        if isinstance(ft[f], list):
            new = list(np.concatenate(ft[f]).flat)
            new.append(ft[f+1])
            ft_flat.append(new)
    return ft_flat
        

ad_ft_flat = flatten(ad_ft)
mci_ft_flat = flatten(mci_ft)
healthy_ft_flat = flatten(healthy_ft)

for i in ad_ft_flat:
    print(len(i))
    print(i)


61
[0.0, 0.0, 3.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.5, 0.0, 0.0, 0.0, 15.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1]
63
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 0.0, 2.0, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0]
67
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.5, 0.0, 4.5, 2.3333333333333335, 13.25, 10.0, 3.3333333333333335, 11.5, 3.0, 0.0, 0.0, 0.0, 13.0, 0.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 3.0, 4.0, 3.0, 3.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 5]
67

In [178]:
def labeler(classic, term):
    blank = []
    for i in classic:
        blank.append(term)
    return blank


ad_labels = labeler(AD_cha_files, "AD")
mci_labels = labeler(MCI_cha_files,"MCI")
healthy_labels = labeler(Healthy_cha_files, "Healthy")

labels = ad_labels + mci_labels + healthy_labels
